# Get BiciMAD data and store it in a database

This Notebooks gets the availbale BiciMAd data from https://opendata.emtmadrid.es/Datos-estaticos/Datos-generales-(1) and stores them in a database.

The following database tables will be removed (if they already exist) and then created:
   * trips
   * situation: stations hourly situation
   
You need to manually download https://datos.madrid.es/egobfiles/MANUAL/208327/bases_bicimad.csv and save it in `data/bases_bicimad.csv`.

In [1]:
pip install pyunpack

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ipynb.fs
from .full.functions import *
import os
import re
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine, text
from sqlalchemy.types import Integer, Text, Float, DateTime, Boolean
import urllib
import zipfile
from pyunpack import Archive
import time

In [3]:
# construct an engine connection string
engine_string = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
    user = 'postgres',
    password = '{password}',
    host = 'postgres-bicimad',
    port = 5432,
    database = 'postgres',
)
# create sqlalchemy engine
engine = create_engine(engine_string)

## Get trips

In [5]:
base_url = "https://opendata.emtmadrid.es"
url = base_url + "/Datos-estaticos/Datos-generales-(1)"
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, "html.parser")
 
urls = []
for link in soup.find_all('a'):
    href = link.get('href')
    if href != None and re.match("^/getattachment/.*(_Usage_Bicimad|_movements|_movements-json).*.aspx$", href):
        urls.append(base_url + href)

extract_dir = "tmp_trips"

In [3]:
for u in urls:
    zip_path, _ = urllib.request.urlretrieve(u)
    with zipfile.ZipFile(zip_path, "r") as f:
        f.extractall(extract_dir)

In [6]:
stations = load_stations('data/bases_bicimad.csv')

In [7]:
sql = text('DROP TABLE IF EXISTS "trips";')
engine.execute(sql)

In [9]:
dtypes = {
    "_id": Text(),
    "user_day_code": Text(),
    "idplug_base": Integer(),
    "user_type": Integer(),
    "idunplug_base": Integer(),
    "travel_time": Integer(),
    "idunplug_station": Integer(),
    "ageRange": Integer(),
    "idplug_station": Integer(),
    "unplug_hourTime": DateTime(),
    "zip_code": Float(),
    "hour": Integer(),
    "day_type": Boolean(),
    "o_dist_km0": Float(),
    "d_dist_km0": Float(),
}

In [10]:
for f in sorted(os.listdir(extract_dir)):
    start = time.time()
    print(extract_dir + '/' + f)
    if f[-5:] == '.json':
        load_trips(stations, extract_dir + '/' + f,
                   calc_ratios=False).to_sql('trips',
                                             engine,
                                             if_exists='append',
                                             dtype=dtypes,
                                             index=False)
    end = time.time()
    print(end - start)

tmp_trips/.ipynb_checkpoints
0.00014448165893554688
tmp_trips/201704_Usage_Bicimad.json
37.42339324951172
tmp_trips/201705_Usage_Bicimad.json
44.70170521736145
tmp_trips/201706_Usage_Bicimad.json
58.63076686859131
tmp_trips/201707_Usage_Bicimad.json
52.60178565979004
tmp_trips/201708_Usage_Bicimad.json
46.71214532852173
tmp_trips/201709_Usage_Bicimad.json
63.4159791469574
tmp_trips/201710_Usage_Bicimad.json
44.50591015815735
tmp_trips/201711_Usage_Bicimad.json
23.508381843566895
tmp_trips/201712_Usage_Bicimad.json
17.07199263572693
tmp_trips/201801_Usage_Bicimad.json
17.503194093704224
tmp_trips/201802_Usage_Bicimad.json
15.570703506469727
tmp_trips/201803_Usage_Bicimad.json
13.14189076423645
tmp_trips/201804_Usage_Bicimad.json
20.191070795059204
tmp_trips/201805_Usage_Bicimad.json
33.924418926239014
tmp_trips/201806_Usage_Bicimad.json
50.87822437286377
tmp_trips/201807_Usage_Bicimad.json
63.397239208221436
tmp_trips/201808_Usage_Bicimad.json
45.299758195877075
tmp_trips/201809_Usage_B

In [14]:
with engine.connect() as con:
    con.execute("""
    DELETE FROM
        trips T1
    USING
        trips T2
    WHERE
        T1."_id" = T2."_id"
        AND T1.ctid < T2.ctid; -- delete the "older" ones
    """)

In [15]:
with engine.connect() as con:
    con.execute('ALTER TABLE trips ADD PRIMARY KEY ("_id");')

In [22]:
# df = pd.read_sql(
#     """SELECT
#         "unplug_hourTime",
#         concat_ws('->', "idunplug_station", "idplug_station") AS od,
#         "travel_time",
#         AVG("travel_time")
#             OVER (
#             PARTITION BY "idunplug_station", "idplug_station"
#             ORDER BY "unplug_hourTime" 
#                   RANGE BETWEEN '28 day' PRECEDING AND current row)
#             AS rolling_average
#     FROM
#         trips
#     LIMIT
#         10000
#     """, engine)
# df

In [23]:
# px.line(df, x='unplug_hourTime', y=['rolling_average'])

In [21]:
with engine.connect() as con:
    con.execute("""
    ALTER TABLE trips 
        ADD COLUMN IF NOT EXISTS "od_time_ratio2" real;
    ALTER TABLE trips 
        ADD COLUMN IF NOT EXISTS "o_time_ratio2" real;
    ALTER TABLE trips 
        ADD COLUMN IF NOT EXISTS "d_time_ratio2" real;
        
    UPDATE trips
        SET
            "od_time_ratio2" = subquery.od,
            "o_time_ratio2" = subquery.o,
            "d_time_ratio2" = subquery.d
        FROM (
            SELECT
                "_id",
                CASE
                    WHEN "travel_time" BETWEEN 1 AND 8 * 3600 THEN
                        "travel_time" / NULLIF(GREATEST(0, AVG("travel_time")
                            OVER (
                            PARTITION BY "idunplug_station", "idplug_station"
                            ORDER BY "unplug_hourTime" 
                                  RANGE BETWEEN '28 day' PRECEDING AND current row)), 0)
                    ELSE
                        NULL
                END
                    AS od,
                CASE
                    WHEN "travel_time" BETWEEN 1 AND 8 * 3600 THEN
                        "travel_time" / NULLIF(GREATEST(0, AVG("travel_time")
                            OVER (
                            PARTITION BY "idunplug_station"
                            ORDER BY "unplug_hourTime" 
                                  RANGE BETWEEN '28 day' PRECEDING AND current row)), 0)
                    ELSE
                        NULL
                END
                    AS o,
                CASE
                    WHEN "travel_time" BETWEEN 1 AND 8 * 3600 THEN
                        "travel_time" / NULLIF(GREATEST(0, AVG("travel_time")
                            OVER (
                            PARTITION BY "idplug_station"
                            ORDER BY "unplug_hourTime" 
                                  RANGE BETWEEN '28 day' PRECEDING AND current row)), 0)
                    ELSE
                        NULL
                END
                    AS d
            FROM
                trips
            ) as subquery
        WHERE
            trips."_id" = subquery."_id"
    """)

## Get statuses

In [4]:
base_url = "https://opendata.emtmadrid.es"
url = base_url + "/Datos-estaticos/Datos-generales-(1)"
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, "html.parser")
 
urls = []
for link in soup.find_all('a'):
    href = link.get('href')
    if href != None and re.match("^/getattachment/.*/(Bicimad_[Ee]?[sS]ta[ct]ions_\d{6}|BiciMAD_stations_20\d{2}_\d_\d|\d{6}(_stations|-json)?).aspx$", href):
        urls.append(base_url + href)

In [5]:
extract_dir = "tmp_situations"

In [6]:
for u in urls:
    zip_path, _ = urllib.request.urlretrieve(u)
    try:
        with zipfile.ZipFile(zip_path, "r") as f:
            f.extractall(extract_dir)
    except zipfile.BadZipFile:
        continue

In [6]:
sql = text('DROP TABLE IF EXISTS "situations";')
engine.execute(sql)

In [7]:
dtypes = {
    "date": DateTime(),
    "activate": Boolean(),
    "name": Text(),
    "reservations_count": Integer(),
    "light": Integer(),
    "total_bases": Integer(),
    "free_bases": Integer(),
    "number": Text(),
    "longitude": Float(),
    "no_available": Integer(),
    "address": Text(),
    "latitude": Float(),
    "dock_bikes": Integer(),
    "id": Integer(),
    "hour": Integer(),
    "day_type": Boolean(),
}

In [8]:
for f in sorted(os.listdir(extract_dir)):
    start = time.time()
    print(extract_dir + '/' + f)
    if f[-5:] == '.json':
        load_situations(extract_dir + '/' + f).to_sql('situations',
                                                      engine,
                                                      if_exists='append',
                                                      index=False,
                                                      dtype=dtypes)
    end = time.time()
    print(end - start)

tmp_situations/202001.json
9.698063611984253
tmp_situations/202002.json
9.119324207305908
tmp_situations/202003.json
9.77554726600647
tmp_situations/202004.json
9.460395097732544
tmp_situations/202005.json
9.856130838394165
tmp_situations/202006_stations.json
9.56312346458435
tmp_situations/202007.json
9.747358083724976
tmp_situations/202008.json
9.879436016082764
tmp_situations/202009.json
9.611745834350586
tmp_situations/202010.json
9.831257104873657
tmp_situations/202011.json
9.827429056167603
tmp_situations/202012.json
11.081334114074707
tmp_situations/202101.json
12.08172345161438
tmp_situations/202102.json
10.77942419052124
tmp_situations/202103.json
12.518405199050903
tmp_situations/Bicimad_Estacions_201809.json
7.693673849105835
tmp_situations/Bicimad_Estacions_201810.json
8.031645774841309
tmp_situations/Bicimad_Stations_201811.json
7.750242233276367
tmp_situations/Bicimad_Stations_201812.json
7.609273910522461
tmp_situations/Bicimad_Stations_201901.json
7.5708630084991455
tmp

## Reading from the db

In [10]:
df = pd.read_sql("SELECT * FROM trips LIMIT 1000", engine)

In [11]:
df

,index,_id,user_day_code,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,zip_code,od_time_ratio,o_time_ratio,d_time_ratio,hour,day_type,o_dist_km0,d_dist_km0
0,0,58e5e5e7865e9c221ca45898,df843a74bba22d2b363e4e96f74b99391276e71dfea92e...,17,1,1,169,41,4,50,2017-03-31 23:00:00+00:00,28005.0,0.670009,0.143761,0.167736,1,0,636.518095,1288.365395
1,1,58e5e5e7865e9c221ca458a7,6a7cf4393301d25f5a2eeaff8c7ba5d406f514ebd086aa...,2,1,14,271,95,5,90,2017-03-31 23:00:00+00:00,28043.0,0.290996,0.262172,0.246034,1,0,1580.506989,1949.659347
2,2,58e5e5e7865e9c221ca458b2,877deab4cc256bb87e02abc1db75005ac2b66d65be8849...,5,1,5,171,15,4,16,2017-03-31 23:00:00+00:00,28053.0,0.737705,0.169862,0.177441,1,0,1117.123486,769.549748
3,3,58e5e5e7865e9c221ca458b8,91bbf6c3ba396f84cce4e96e59be42b0824a16c8a1bf89...,18,1,20,388,23,4,57,2017-03-31 23:00:00+00:00,28012.0,0.388014,0.370689,0.362608,1,0,551.058095,1635.858302
4,4,58e5e5e7865e9c221ca458b9,4476cabb023b8b4473e5b6e78a1a79e0ebc8910c8e9989...,2,1,2,381,23,4,43,2017-03-31 23:00:00+00:00,28042.0,0.581058,0.364001,0.353920,1,0,551.058095,731.321696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,58e5e5ed865e9c221ca45d0a,4843b8e8a0587a1745305d5fc58066408480383ebfc240...,1,1,10,390,99,4,45,2017-04-01 03:00:00+00:00,28012.0,0.729881,0.405945,0.451409,5,0,2746.394035,1237.860197
996,996,58e5e5ed865e9c221ca45d0c,b8953f56422257e71ca31b5c347dfeb7cb42ca4fac12a3...,3,1,12,210,173,3,138,2017-04-01 03:00:00+00:00,28002.0,0.083129,0.184534,0.160503,5,0,2049.725535,3393.410018
997,997,58e5e5ed865e9c221ca45d0d,17031713ced93bcea71352ed10899dd9e6ccdb252514b1...,14,1,6,763,74,5,135,2017-04-01 03:00:00+00:00,28037.0,0.579787,0.695482,0.510137,5,0,2375.574276,2915.287342
998,998,58e5e5ed865e9c221ca45d0e,ae009ae55b6542d112edca55fa5cb772c4d5d6b7d4178a...,16,3,3,572,81,4,171,2017-04-01 03:00:00+00:00,NaN,0.987058,0.442372,0.461608,5,0,1274.381217,1924.914926
